In [1]:
#данный скрипт:
## - высчитывает еженедельные чарты стримингов, усредняя ежедневные чарты за 7 дней 
## - стриминги: Apple Music, VK, Deezer, Yandex
## - должен запускаться один раз в неделю ранним утром пятницы после Weekly_parsing.py

## соединяет получающиеся чарты в единый html файл для публикации на сайте (включая "настоящие" еженедельные чарты)

#на выходе:
## - обновляет csv файлы с соответствующими еженедельными чартами 4-x стримингов
## - сохраняет в корневую директорию html файлы с новыми еженедельными чартами

In [2]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import datetime
from datetime import datetime, date, time, timezone
from dateutil.relativedelta import relativedelta

In [33]:
#задаем команду для получения даты
currentDT = datetime.datetime.now() 

In [ ]:
# технический код, созданный для момента, когда не было исходных файлов в директории
#for i in ["vk", "apple", "deezer", "yandex"]:
#    cols = ['rank', 'title', 'artist', "week", "raw_rank"]
#    name = "all_" + i + "_weekly.csv" 
#    pd.DataFrame(columns = cols).to_csv(name, encoding= "utf-8")

In [181]:
#загружаем полные базы данных по всем ежедневным чартам
all_vk = pd.read_csv("all_vk.csv")
all_yandex = pd.read_csv("all_yandex.csv")
all_deezer = pd.read_csv("all_deezer.csv")
all_apple = pd.read_csv("all_apple.csv")

#удаляем получающуюся после импорта лишнюю колонку 
all_charts= [all_apple, all_deezer, all_vk, all_yandex]
for i in all_charts:
    i.drop(i.columns[[0]], axis=1, inplace=True)

In [191]:
# функция для получения недельного чарта через усреднение ежедневных

#заменяем номер строчки на X в те дни, когда трека нет в чарте 
# X в будущем должен оцениваться с использованием данных других стримингов (особенно спотифай)
## пока что X - константа.

def average(df, X):
    
    df['Datetime'] = [datetime.strptime(i, "%d/%m/%Y") for i in df["date"]]
    
    #take last week only
    date_start = currentDT - relativedelta(days=+7)
    date_end = currentDT - relativedelta(days=+1)
    last_week_df = df[date_start <= df["Datetime"]]
    df = last_week_df
    
    raw_rank = []
    songs =[]
    artists = []
    for i in list(set(df["title"])):
        newdf = df[df["title"]==i]
        for j in list(set(newdf["artist"])):
            one_track_df = newdf[newdf["artist"]==j]
            #how many missing days?
            miss_days = 7 - len(one_track_df)
            average_rank = (sum(one_track_df["rank"]) + miss_days*(X)) / 7  #считаем среднюю строчку трека (использование X)
            songs.append(i)
            artists.append(j)
            raw_rank.append(average_rank)

    data = {"raw_rank": raw_rank, "title": songs, "artist": artists}        
    new_chart = pd.DataFrame(data)
    new_chart.sort_values(by=['raw_rank'], inplace=True)

    new_chart['rank'] = new_chart.reset_index().index +1
    new_chart.reset_index(inplace=True)
    week = datetime.strftime(date_start,"%d/%m/%y") + " - " + datetime.strftime(date_end,"%d/%m/%y")
    new_chart["week"] = week
    
    new_chart = new_chart[['rank', 'title', 'artist', "week", "raw_rank"]]
    
    #округляем raw_rank
    new_chart["raw_rank"] = round(new_chart["raw_rank"], 3)
    
    return new_chart

In [168]:
#просто техническая функция для отображения изначальных имен чартов
def name_of_global_obj(xx):
    return [objname for objname, oid in globals().items()
            if id(oid)==id(xx)][0]

In [196]:
#выполняем функцию average() и обновляем имеющиеся еженедельные чарты из csv в корне

for c in all_charts:
    output_chart = average(c, 150)
    name_of_chart = str(name_of_global_obj(c)) 
    
    #сохраняем чарт в HTML
    platform_name = name_of_chart.replace("all_", "")
    html_name = "current_"+platform_name+"_html.html"
    output_chart.to_html(html_name, encoding = "utf-8")
    
    #переходим к загрузке имеющегося csv c предыдущими еженедельными чартами
    name_of_weekly_chart = name_of_chart +"_weekly.csv"
    old_csv = pd.read_csv(name_of_weekly_chart)
    
    old_csv = old_csv.drop(old_csv.columns[[0]], axis=1) #удаляем получающуюся после импорта лишнюю колонку 
    frames = [old_csv, output_chart]
    new_csv = pd.concat(frames, sort=False)
    new_csv.to_csv(name_of_weekly_chart, encoding = "utf-8")

### создание финального HTML

In [209]:
l = []

h1 = pd.DataFrame({'Spotify Weekly Top 200' : '-'}, index=[0])
h2 = pd.DataFrame({'Youtube Top 100 Tracks' : '-'}, index=[0])
h3 = pd.DataFrame({'Apple Music Top 100 -- by IMI' : '-'}, index=[0])
h4 = pd.DataFrame({'Yandex Music Top 100 -- by IMI' : '-'}, index=[0])
h5 = pd.DataFrame({'Deezer Top 100 -- by IMI' : '-'}, index=[0])
h6 = pd.DataFrame({'VK/BOOM Top 100 -- by IMI' : '-'}, index=[0])

In [210]:
#открываем html спотифая и ютуба из корневой директории (туда они сохранены weekly_parsing.py)
l.append(h1.to_html())
with open('current_spotify_html.html', 'r') as f:
    l.append(f.read())
    
l.append(h2.to_html())
with open('current_youtube_html.html', 'r') as f:
    l.append(f.read())
    
l.append(h3.to_html())
with open('current_apple_html.html', 'r') as f:
    l.append(f.read())
    
l.append(h4.to_html())
with open('current_yandex_html.html', 'r') as f:
    l.append(f.read())
    
l.append(h5.to_html())
with open('current_deezer_html.html', 'r') as f:
    l.append(f.read())
    
l.append(h6.to_html())
with open('current_vk_html.html', 'r') as f:
    l.append(f.read())

In [ ]:
#сохраняем итоговый html
final_html = ''.join(l)
with open("current_full_html.html", "w", encoding="utf-8") as file:
    file.write(final_html)